# Rouge

In [ ]:
%cd /content/

/content


In [ ]:
!git clone https://github.com/pltrdy/rouge
%cd rouge
!python setup.py install
#!pip install rouge

In [ ]:
%cd /content/rouge
from rouge import Rouge 

/content/rouge


In [ ]:
%cd
%cd "/content/drive/Shared drives/rs/RSProject/recipe-personalization_org/OUTPUT50KTchPrfNew"

/root
/content/drive/Shared drives/rs/RSProject/recipe-personalization_org/OUTPUT50KTchPrfNew


# BLEU

In [ ]:

def calc_bleu (filepath):
  
  #filepath = 'model_baseline_e3/test_i94035_u24751/output.txt'

  nlp = spacy.load("en", disable=["parser", "tagger", "textcat", "ner", "vectors"])
  bleu_1 = []
  bleu_2 = []
  bleu_3 = []
  bleu_4 = []
  with open(filepath) as fp:
    lines = list(fp)
    for line in lines:
      
      if "Original Steps:" in line:
          ref_text = lines[lines.index(line)+1]
          reference_tokens = [t.text for t in nlp(ref_text.strip())]
          
          #print (reference_tokens)
      if "MODEL OUTPUT:" in line:
          gen_text = lines[lines.index(line)+1]
          generated_tokens = [t.text for t in nlp(gen_text.strip())]
          #print (generated_tokens)
  bleu_1.append(sentence_bleu([reference_tokens], generated_tokens, weights=(1, 0, 0, 0)))
  bleu_2.append(sentence_bleu([reference_tokens], generated_tokens, weights=(0.5, 0.5, 0, 0)))
  bleu_3.append(sentence_bleu([reference_tokens], generated_tokens, weights=(0.33, 0.33, 0.33, 0)))
  bleu_4.append(sentence_bleu([reference_tokens], generated_tokens, weights=(0.25, 0.25, 0.25, 0.25)))

  final_bleu = [bleu_1,bleu_2,bleu_3,bleu_4]
  return final_bleu


#print (final_blue)    
#print(' \nMean BLEU-1 score: {:.3f}\nMean BLEU-2 score: {:.3f}\nMean BLEU-3 score: {:.3f}\nMean BLEU-4 score: {:.3f}'.format(np.array(bleu_1).mean()*100, np.array(bleu_2).mean()*100, np.array(bleu_3).mean()*100, np.array(bleu_4).mean()*100))


*rouge* and *bleu* calculation over output folder

In [ ]:
import os
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu
import spacy
import numpy as np
##################### need to change the model 
rootdir = 'model_tech_pref_e2'
files_list = []
c = 0

for subdir, dirs, files in os.walk(rootdir):
  for file in files:

    if file.endswith('.txt'):
      c+=1
      if (c%1000 == 0): print (c)

      name = os.path.join(subdir, file)
      files_list.append(name)

df1 = pd.DataFrame(files_list)
df1.to_pickle("/content/drive/My Drive/ForMiri (1)/file_list_model_tech_pref_e2.pkl")      
print ("finish files list")
count = 0
rootdir = 'model_tech_pref_e2'
scores_list = []
for file in files_list:
  count +=1 
  if (count%1000 == 0): print (count)
  fp = open(file,'r')
  lines = list(fp)
  for line in lines:
    if "Original Steps:" in line:
      original = lines[lines.index(line)+1]
       #print (original)
    if "MODEL OUTPUT:" in line:
      model = lines[lines.index(line)+1]
        # print (model)

  rouge = Rouge()
  scores = rouge.get_scores(original, model, avg=True)
  scores_list.append(scores)

df = pd.DataFrame(scores_list)
df.to_pickle("/content/drive/My Drive/ForMiri (1)/model_user-pref_e2_rouge_score.pkl")
print ("finish rouge")

count = 0
bleus = []
scores_list2 = []
for file in files_list:  
  b_score = calc_bleu (file)
  scores_list2.append(b_score)
  count +=1
  if (count % 1000 ==0): print (count)

df = pd.DataFrame(scores_list2)
#df.head(10)
df.to_pickle("/content/drive/My Drive/ForMiri (1)/model_tech_pref_e2_bleu_score.pkl")

In [ ]:
new_dict.head(10)

,rouge-1,rouge-2,rouge-l
0,"{'f': 0.27586206539387637, 'p': 0.179775280898...","{'f': 0.11304347471417782, 'p': 0.073446327683...","{'f': 0.810344824014566, 'p': 0.52808988764044..."
1,"{'f': 0.523489927887933, 'p': 0.51315789473684...","{'f': 0.21768706983201452, 'p': 0.213333333333...","{'f': 1.087248317149678, 'p': 1.06578947368421..."
2,"{'f': 0.18390804105694292, 'p': 0.163265306122...","{'f': 0.0, 'p': 0.0, 'r': 0.0}","{'f': 0.25287355829832214, 'p': 0.224489795918..."
3,"{'f': 0.4827586158382878, 'p': 0.4117647058823...","{'f': 0.22222221739369008, 'p': 0.1875, 'r': 0...","{'f': 0.4827586158382878, 'p': 0.4117647058823..."
4,"{'f': 0.43333332844583333, 'p': 0.509803921568...","{'f': 0.16949152054007485, 'p': 0.2, 'r': 0.14...","{'f': 0.7999999951125, 'p': 0.9411764705882353..."
5,"{'f': 0.2093023224094105, 'p': 0.1304347826086...","{'f': 0.023529408635986578, 'p': 0.01459854014...","{'f': 0.4186046479908058, 'p': 0.2608695652173..."
6,"{'f': 0.24444443980000005, 'p': 0.192982456140...","{'f': 0.022727268099174493, 'p': 0.01785714285...","{'f': 0.3999999953555556, 'p': 0.3157894736842..."
7,"{'f': 0.3833333283347223, 'p': 0.3770491803278...","{'f': 0.10924369248040415, 'p': 0.107438016528...","{'f': 1.299999995001389, 'p': 1.27868852459016..."
8,"{'f': 0.29230768745088764, 'p': 0.25, 'r': 0.3...","{'f': 0.031249995147705833, 'p': 0.02666666666...","{'f': 0.6615384566816568, 'p': 0.5657894736842..."
9,"{'f': 0.213333328376889, 'p': 0.23529411764705...","{'f': 0.0, 'p': 0.0, 'r': 0.0}","{'f': 0.3199999950435556, 'p': 0.3529411764705..."


In [ ]:
new_dict['rouge-lr'] = [g['r'] for g in new_dict['rouge-l']]
new_dict['rouge-lp'] = [g['p'] for g in new_dict['rouge-l']]
new_dict['rouge-l'] = [g['f'] for g in new_dict['rouge-l']]


In [ ]:
print (new_dict['rouge-l'].mean())
print (new_dict['rouge-lp'].mean())
print (new_dict['rouge-lr'].mean())

0.723264988611434
0.7042017765734079
0.9261146777944983


In [ ]:
print (new_dict['rouge-l'].mean())
print (new_dict['rouge-lp'].mean())
print (new_dict['rouge-lr'].mean())

0.709141081344976
0.6976582404921252
0.9041960073314946


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
import spacy
import numpy as np

reference = [['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']]
candidate = ['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog', 'from', 'space']
score = sentence_bleu(reference, candidate)
print(score)


0.7860753021519787
